<a href="https://colab.research.google.com/github/stwagner07/atr-historical-research/blob/main/colab-notebooks/colab_textcorrection_ner_nlp_personal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## OCR correction and Named Entity Recognition (NER) with PySpellChecker and Stanza

This script offers a more traditional NLP alternative to using LLMs for OCR post-processing for Python users. The packages used here as smaples are PySpellChecker and Stanza. But they are by no means the only options.




In [1]:
# Install the packages needed

!pip install pyspellchecker stanza
import requests
import stanza
from spellchecker import SpellChecker

print("Installation successful!")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 25.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 22.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 51.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 43.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 23.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 77.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 590.6

### Download dictionary for spell checking

The challenge here is to find a dictionary for your language that is comprehensive enough and, in historical research, also captures old word froms. Unfortunately, an extensive German dictionary I used in the past is no longer available. For testing purposes, we are now working with a more limited German word list from Github.

In [2]:
# download German word list provided by Marvin J. Wendt
dwds_url = "https://gist.githubusercontent.com/MarvinJWendt/2f4f4154b8ae218600eb091a5706b5f4/raw/36b70dd6be330aa61cd4d4cdfda6234dcb0b8784/wordlist-german.txt"
response = requests.get(dwds_url)

if response.status_code == 200:
    # Save the word list as a local file
    with open("word_list.txt", "w", encoding="utf-8") as file:
        file.write(response.text)
    print("Word list successfully downloaded.")
else:
    print("Failed to download word list.")

# change SpellChecker language from English to German

spell = SpellChecker(language=None)
spell.word_frequency.load_text_file("word_list.txt")
print("German word list loaded into PySpellChecker.")

Word list successfully downloaded.
German word list loaded into PySpellChecker.


### Correct OCR errors

In [3]:
# function for error correction with PySpellChecker
# note that the original word will be used when word not in word list

def correct_ocr_errors(text):
    words = text.split()
    corrected_text = " ".join([spell.correction(word) if spell.correction(word) else word for word in words])
    return corrected_text

### Read sample text from GitHub

In [ ]:
# Here you can load one of the samples provided in the atr-historical-research repository
# or load raw text from your own repository

github_raw_url = "https://raw.githubusercontent.com/stwagner07/atr-historical-research/refs/heads/main/sample_data_txt/ThS.L.V.413u14Text.txt"
response = requests.get(github_raw_url)
if response.status_code == 200:
    input_text = response.text
    corrected_text = correct_ocr_errors(input_text)
    print("Corrected OCR Text:", corrected_text[:700])  # Print first 700 characters
else:
    print("Failed to access sample text.")

# Be patient! Code execution can take several minutes as checking each token against the word list is not the fastest approach.
# Here, a regex-based replacement of false characters or character combinations could be beneficial, but building a suitable regex can take time to build.

Corrected OCR Text: aa pa 1 (#9) ################################################ deutsche kolonialzeitung Organ der deutschen kolonialgesellschaft Die Deutsche Kolonialzeitung erscheint vierwöchentlich. - redakteur Gustav reinecke - Alle Sendungen für die Redaktion ad Expedition diebes Blattes sind zu richten an die Adreffe: Deutsche kolonialgeſellſchaft, Berlin wal Linkſtraße 25. nr 1592 der Postzeitungsliste - oder im buchhandel jährlich Bezugspreis in deutschland und Österreich-Ungarn durch die Post Als Jahresbeitrag find in deutschland und 8 marta im auslade jährlich 10 Marl.—Die Mitglieder der Deutschen kolonialgesellschaft erhalten die Zeitung unentgeltlich Beitrittserklärungen zur Deutschen kolonialgese


### Named Entity Recognition (NER) with Stanza

In [ ]:
stanza.download("de")  # download German model
nlp = stanza.Pipeline(lang="de", processors="tokenize,ner")

INFO:stanza:Downloaded file to /root/stanza_resources/resources.json
INFO:stanza:Downloading default packages for language: de (German) ...


INFO:stanza:Downloaded file to /root/stanza_resources/de/default.zip
INFO:stanza:Finished downloading models and saved to /root/stanza_resources
INFO:stanza:Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


INFO:stanza:Downloaded file to /root/stanza_resources/resources.json
INFO:stanza:Loading these models for language: de (German):
| Processor | Package      |
----------------------------
| tokenize  | combined     |
| mwt       | combined     |
| ner       | germeval2014 |

INFO:stanza:Using device: cpu
INFO:stanza:Loading: tokenize
INFO:stanza:Loading: mwt
INFO:stanza:Loading: ner
INFO:stanza:Done loading processors!


### Extract named entities

In [ ]:
# function for NER with focus on persons and locations

def extract_named_entities(text):
    doc = nlp(text)
    named_entities = {"Person": [], "Location": []}
    for ent in doc.ents:
        if ent.type == "PER":
            named_entities["Person"].append(ent.text)
        elif ent.type in ["LOC", "GPE"]:
            named_entities["Location"].append(ent.text)
    return named_entities

### Apply NER to corrected text from the previous step

In [ ]:
# This generates output in JSON format.

entities = extract_named_entities(corrected_text)
print("Named Entities:", entities)

Named Entities: {'Person': ['Gustav reinecke', 'Carl hermanns', 'werther', 'Georg von kaufmann', 'pa Hall', 'Hendrik Witbooi', 'Adamaua', 'vb Scheele', 'Wilhelmsland', 'amin paschas'], 'Location': ['deutsche', 'deutschen', 'Deutsche', 'Berlin', 'deutschland', 'Österreich', 'Ungarn', 'deutschland', 'Marl', 'deutschen', 'Österreich', 'Ungarn', 'Berlin', 'Mauerstraße 44', 'Usambara', 'Ostafrikas', 'Nkonya', 'Ntshumuru', 'Frankfurt aa ma', 'deutscher', 'Bremen', 'Angra-Pequena', 'deutsche', 'Bremer', 'Deutsche Reich', 'ostafrika', 'Südsee', 'deutsche', 'Kamerun', 'deutschen', 'zentralafrikanischen', 'Tschadsee', 'sambesi', 'Afrika', 'südwestafrika', 'deutsche', 'deutschen', 'Kameruns', 'Franzosen', 'Engländern', 'Tschadsee', 'Ostafrika', 'jaffa', 'tanganjika', 'Norddeutschen Lloyd', 'Deutschland', 'deutschen']}


**Note**: This is a sample script for testing purposes which still needs updates to be fully functional for research. Especially when you are processing sensitive text, you should consider using a local installation of Python or running code in an institutional environment.